# This notebook only extract graph, text, layout from PDF files

### Read PDF Data

In [1]:
PDF_File_Path="../Data/PDF"
PDF_Extract_Output_Path="../Data/PDF-Extract"
print("PDF file path: ", PDF_File_Path)

PDF file path:  ../Data/PDF


### The sample of text extraction (only 1 pdf)

In [ ]:
import fitz  # pip install PyMuPDF
import os
import base64

# def extract_with_layout(pdf_path, output_dir="output"):
#     os.makedirs(output_dir, exist_ok=True)
#     doc = fitz.open(pdf_path)
#
#     for page_index, page in enumerate(doc, start=1):
#         # 获取结构化内容，包括文本块与图像块
#         pagedict = page.get_text("dict", sort=True)
#         items = pagedict["blocks"]
#
#         # 保存为 HTML，能直观保留文字布局与图片
#         html = page.get_text("html")
#         with open(os.path.join(output_dir, f"page{page_index}.html"), "w", encoding="utf-8") as f:
#             f.write(html)
#
#         # 遍历块，识别图像和文字
#         for block in items:
#             if block["type"] == 1:  # 图像块
#                 img = block["image"]
#                 ext = block.get("ext", "png")
#                 x0, y0, x1, y1 = block["bbox"]
#                 img_filename = os.path.join(output_dir, f"page{page_index}_img_{int(x0)}_{int(y0)}.{ext}")
#                 with open(img_filename, "wb") as imgf:
#                     imgf.write(img)
#                 print(f"Page {page_index}: image saved → {img_filename}")
#             elif block["type"] == 0:  # 文本块
#                 text = block["lines"]
#                 bbox = block["bbox"]
#                 print(f"Page {page_index} text block at {bbox}:")
#                 for line in text:
#                     spans = line["spans"]
#                     for span in spans:
#                         print(span["text"], end="")
#                 print("\n")
import torch
import base64
import urllib.request

from io import BytesIO
from PIL import Image
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration

from olmocr.data.renderpdf import render_pdf_to_base64png
from olmocr.prompts import build_finetuning_prompt
from olmocr.prompts.anchor import get_anchor_text

# Initialize the model
model = Qwen2VLForConditionalGeneration.from_pretrained("allenai/olmOCR-7B-0225-preview", torch_dtype=torch.bfloat16).eval()
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Grab a sample PDF
# urllib.request.urlretrieve("https://molmo.allenai.org/paper.pdf", "./paper.pdf")

# Render page 1 to an image
image_base64 = render_pdf_to_base64png("./Data/PDF/2023-CSR-report_e.pdf", 1, target_longest_image_dim=1024)

# Build the prompt, using document metadata
anchor_text = get_anchor_text("./Data/PDF/2023-CSR-report_e.pdf", 1, pdf_engine="pdfreport", target_length=4000)
prompt = build_finetuning_prompt(anchor_text)

# Build the full prompt
messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}},
                ],
            }
        ]

# Apply the chat template and processor
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
main_image = Image.open(BytesIO(base64.b64decode(image_base64)))

inputs = processor(
    text=[text],
    images=[main_image],
    padding=True,
    return_tensors="pt",
)
inputs = {key: value.to(device) for (key, value) in inputs.items()}


# Generate the output
output = model.generate(
            **inputs,
            temperature=0.8,
            max_new_tokens=50,
            num_return_sequences=1,
            do_sample=True,
        )

# Decode the output
prompt_length = inputs["input_ids"].shape[1]
new_tokens = output[:, prompt_length:]
text_output = processor.tokenizer.batch_decode(
    new_tokens, skip_special_tokens=True
)

print(text_output)
# ['{"primary_language":"en","is_rotation_valid":true,"rotation_correction":0,"is_table":false,"is_diagram":false,"natural_text":"Molmo and PixMo:\\nOpen Weights and Open Data\\nfor State-of-the']


C:\Users\Tommy\anaconda3\envs\MultiModel_Analysis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# extract_with_layout("../Data/PDF/2023-CSR-report_e.pdf")
